In [ ]:
#Alexandra Rivera
#Carolina Campos

In [1]:
!pip install gradio

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/50.9 kB ? eta -:--:--
     -------------------------------- ------- 41.0/50.9 kB 2.0 MB/s eta 0:00:01
     -------------------------------- ------- 41.0/50.9 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 50.9/50.9 kB 437.1 kB/s eta 0:00:00
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.2 MB 5.5 MB/s eta 0:00:03
    --------------------------------------- 0.2/12.2 MB 2.6 MB/s eta 0:00:05
   - -------------------------------------- 0.4/12.2 MB 3.0 MB/s eta 0:00:04
   - -------------------------------------- 0.6/12.2 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/12.2 MB 4.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/12.2 MB 5.3 MB/s et

In [2]:
import gradio as gr
import openai
import pandas as pd
import tiktoken
import ast
from openai.embeddings_utils import get_embedding, distances_from_embeddings

In [3]:
openai.api_key = ''

In [4]:
class EmbeddingChatbot():
    def __init__(self, 
    	embeddings_path, 
    	model_embeddings = "text-embedding-ada-002",
    	model_completion = "gpt-3.5-turbo-instruct",
    	max_prompt_tokens = 1200,
    	max_answer_tokens = 600):
        self.__df = pd.read_csv('embeddings_txt.csv')
        self.__df.embeddings = self.__df.embeddings.apply(ast.literal_eval)
        self.model_embeddings = model_embeddings
        self.model_completion = model_completion
        self.max_prompt_tokens = max_prompt_tokens
        self.max_answer_tokens = max_answer_tokens 
    
    def get_rows_sorted_by_relevance(self, question, df):
        """
        Function that takes in a question string and a dataframe containing
        rows of text and associated embeddings, and returns that dataframe
        sorted from least to most relevant for that question
        """
    
        # Get embeddings for the question text
        question_embeddings = get_embedding(question, engine=self.model_embeddings)
    
        # Make a copy of the dataframe and add a "distances" column containing
        # the cosine distances between each row's embeddings and the
        # embeddings of the question
        df_copy = df.copy()
        df_copy["distances"] = distances_from_embeddings(
        	question_embeddings,
        	df_copy["embeddings"].values,
        	distance_metric="cosine"
        	)
    
        # Sort the copied dataframe by the distances and return it
        # (shorter distance = more relevant so we sort in ascending order)
        df_copy.sort_values("distances", ascending=True, inplace=True)
        return df_copy


    def create_prompt(self, question, df, max_token_count):
        """
        Given a question and a dataframe containing rows of text and their
        embeddings, return a text prompt to send to a Completion model
        """
        # Create a tokenizer that is designed to align with our embeddings
        tokenizer = tiktoken.get_encoding("cl100k_base")
    
        # Count the number of tokens in the prompt template and question
        prompt_template = """
        Answer the question based on the context below, and if the question
        can't be answered based on the context, say "I don't know"
        
        Context:

            {}
        
        ---

        Question: {}
        Answer:"""
    
        current_token_count = len(tokenizer.encode(prompt_template)) + \
                                len(tokenizer.encode(question))
    
        context = []
        for text in self.get_rows_sorted_by_relevance(question, df)["text"].values:
        
            # Increase the counter based on the number of tokens in this row
            text_token_count = len(tokenizer.encode(text))
            current_token_count += text_token_count
        
            # Add the row of text to the list if we haven't exceeded the max
            if current_token_count <= max_token_count:
                context.append(text)
            else:
                break

        return prompt_template.format("\n\n###\n\n".join(context), question)


    def answer_question(self, question, df, max_prompt_tokens, max_answer_tokens):
        """
        Given a question, a dataframe containing rows of text, and a maximum
        number of desired tokens in the prompt and response, return the
        answer to the question according to an OpenAI Completion model
    
        If the model produces an error, return an "I do not know"
        """

        prompt = self.create_prompt(question, df, max_prompt_tokens)
        try:
            response = openai.Completion.create(
                model=self.model_completion,
                prompt=prompt,
                max_tokens=self.max_answer_tokens
            )
            return response["choices"][0]["text"].strip()
        except Exception as e:
            print(e)
            return ""

    def predict(self, question, history):
    	return self.answer_question(question, self.__df, self.max_prompt_tokens, self.max_answer_tokens)

In [5]:
bot = EmbeddingChatbot(
    embeddings_path='embeddings_txt.csv',
    model_embeddings = 'text-embedding-ada-002',
    model_completion ='gpt-3.5-turbo-instruct',
    max_prompt_tokens = 1800,
    max_answer_tokens = 600
    )

In [6]:
gr.ChatInterface(bot.predict).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
